# CNN

* Créer des paires de document (article, highlights)
* Suppression de la ponctuation
* Séparation en token en minuscules
* Suppression des stopwords pour les articles
* Calcul des fréquences et tf-idf sur les deux types de documents
* Enregistrement du nouveau jeu de données d’entraînement pour usage ultérieur

Vous êtes allé jusqu'au bout ? Vérifiez-bien que vous avez les éléments suivants :

* le texte final est tokenisé sans ponctuation et en minuscules à l’aide d’une regex ;
* tous les documents ont été transformés en matrices de valeurs tf-idf calculées sur les paires (articles, highlight).

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from os import getcwd, listdir
from os.path import isfile, join
import bs4

In [6]:
# ---------------------------------------------------------------------------------------------
#                               MAIN
# ---------------------------------------------------------------------------------------------
verbose = False
verboseMain = False

# Récupère le répertoire du programme
file_path = getcwd() + "\\"
data_path = file_path+"cnn2\\"
data_path

In [46]:
fichiers = [f for f in listdir(data_path) if isfile(join(data_path, f)) and f.endswith(".html")]
fichiers

['000c835555db62e319854d9f8912061cdca1893e.html',
 '00a51d5454f2ef7dbf4c53471223a27fb9c20681.html']

In [13]:
# Etape 2 : utiliser le package BeautifulSoup
# qui "comprend" les balises contenues dans la chaine de caractères renvoyée par la fonction request
def get_page(url):
    html = open(url).read()
    return bs4.BeautifulSoup(html, "html.parser")

In [41]:
def get_data_from_page(file_path):
    page = get_page(file_path)
    titre = ""
    try:
        titre = page.findChild('h2', {'class': 'cd__headline'}).getText()
    except:
        titre = page.findChild('h1').getText()
        
    highlights = []
    try:
        #html_hight = page.findChild('div', {'class': 'el__storyhighlights'})
        for ul in page.findAll('li', {'class': 'el__storyhighlights__item el__storyhighlights--normal'}):
            highlights.append(ul.getText())
    except:
        for ul in page.findAll('li'):
            highlights.append(ul.getText())

    # div l-container
    div1 = page.find('section', {'id': 'body-text'})
    div2 = div1.find('div', {'class': 'l-container'})

    sentences = ""

    for stc in div2.findAll('p', {'class': 'zn-body__paragraph'}):
        sentences += stc.getText().strip() + " "
    
    return (titre, highlights, sentences)

In [47]:
df_origin = pd.DataFrame(columns=['titre','highlights','sentences'])

In [48]:
for file in fichiers:
    try:
        titre, highlights, sentences = get_data_from_page(data_path+file)
        new_row = {'titre':titre, 'highlights':highlights, 'sentences':sentences}
        df_origin = df_origin.append(new_row,ignore_index=True, verify_integrity=False)
    except:
        pass
print(df_origin.shape)
df_origin.head()

(2, 3)


,titre,highlights,sentences
0,Pope Francis names new slate of cardinals,[The 15 new cardinals will be installed on Feb...,"(CNN)For the second time during his papacy, Po..."
1,Sen. Marco Rubio announces presidential run,"[Raul Reyes: In seeking Latino vote, Marco Rub...",Raul A. Reyes is an attorney and member of the...
